In [1]:
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
from datasets import load_dataset
from transformers import set_seed

In [2]:

seed = 42
set_seed(seed)






# Corrección: cargar JSON local correctamente
dataset = load_dataset('json', data_files='./data/discourse_qa.json')
#remove columns that are not needed "question" and "answer"
dataset = dataset.remove_columns(["question", "answer"])
#split train and test
dataset = dataset["train"].train_test_split(0.1)

In [4]:
model_name = "PabloCano1/llama-3-1b-1era-prueba"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             attn_implementation='eager',
                                             device_map="cpu",dtype=torch.bfloat16)
model.to("cuda")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/834 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

RuntimeError: Error(s) in loading state_dict for Embedding:
	size mismatch for weight: copying a param with shape torch.Size([128256, 2048]) from checkpoint, the shape in current model is torch.Size([128256, 3072]).

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# ejemplos planos
# predictions = ["respuesta generada por el modelo 1", "respuesta generada 2"]
# reales = ["respuesta real 1", "respuesta real 2"]

reales = [i['text'] for i in dataset['test']]


BLEU: 0.8442136252749054
ROUGE: {'rouge1': np.float64(0.8333333333333333), 'rouge2': np.float64(0.5), 'rougeL': np.float64(0.8333333333333333), 'rougeLsum': np.float64(0.8333333333333333)}


In [ ]:

# Para BLEU la librería espera referencias como lista de listas (cada pred puede tener varias refs)
references_for_bleu = [[r] for r in reales]   # [["respuesta real 1"], ["respuesta real 2"]]

# Opción 1: pasar cadenas y dejar que evaluate haga el tokenizado internamente
bleu_result = bleu.compute(predictions=predictions, references=references_for_bleu)
print("BLEU:", bleu_result["bleu"])

# ROUGE normalmente recibe listas de strings
rouge_result = rouge.compute(predictions=predictions, references=reales)
print("ROUGE:", rouge_result)